In [3]:
import math
import random
import numpy as np

In [4]:
# def generate_point_in_circle(radius):
#     angle = random.uniform(0, 2 * math.pi)
#     r = radius * math.sqrt(random.uniform(0, 1))
#     x = r * math.cos(angle)
#     y = r * math.sin(angle)
#     return x, y

def generate_STA_distance(radius):
    angle = random.uniform(0, 2 * math.pi)
    r = radius * math.sqrt(random.uniform(0, 1))
    x = r * math.cos(angle)
    y = r * math.sin(angle)
    return math.sqrt(x**2 + y**2)

In [5]:
# sta_amount: Total contend stations number
# RA_RU: RA_RU number reserved for UORA
# Lmax: Retransmit limit
# TXOP: Transaction Opportunity (time length in ms)
# ocw_min: minimum OCW
# ocw_max: maximum OCW
# CR: Capture Ratio (CR=infinite = no capture, CR=0 = always capture)

def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, CR, step, gamma, r):
    total_power = 0
    counter_delay_slot=0
    success_sta=0
    # setting ocw for each transmission [ocw_min, 1st_retry_ocw...ocw_max]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # Imax: Reserved slot for transmission (total slots in worst case)
    Imax = 0
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)
    
    # M_i[1]
    M_i_1  = [ 0 for _ in range(0, Imax)]
    M_i_s  = [ 0 for _ in range(0, Imax)]
    M_i_f  = [ 0 for _ in range(0, Imax)]


    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter, obo_counter[STA1_obo_counter, STA2_obo_counter...]
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time, retrans_counter[STA1_obo_counter, STA2_obo_counter...]
    retrans_counter = [0 for _ in range(0, sta_amount)]
    # STA's distance from AP (From 0~1)
#     sta_distance = [random.uniform(0, 1) for _ in range(0, sta_amount)]
    sta_distance = [generate_STA_distance(r) for _ in range(0, sta_amount)]    
    # record STAs choosen RA-RU, sta_on_R[slot_number][STA_1, STA_26...]
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    # Accumulated station transmit power usage
    sta_power_usage = [0 for _ in range(0, sta_amount)]
    
    capture_power_ratio = 10 ** (CR / 10)

    
    # No.i slot    
    for i in range(0, Imax):
        # STA's Transmit power [1P, 2P, 4P, 8P]
#         sta_power = [2 ** random.randint(0, 3) for _ in range(0, sta_amount)]
        sta_power = [2 ** (x if x <= (step - 1) else (step - 1)) for x in retrans_counter]

        # No.x STA
        for x in range (0,sta_amount):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < Lmax):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
                
                # record power usage
                sta_power_usage[x] += sta_power[x]
                # record M_i[1]
                if retrans_counter[x] == 0:
                    M_i_1[i] += 1
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # Record power (no power control so always 1)
            for sta_index in sta_on_R[i][r]:
                total_power += sta_power[sta_index]
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record access delay slot number
                counter_delay_slot += (i+1)
                success_sta += 1
                
                M_i_s[i] += 1
#                 total_power += sta_power_usage[success_sta_index]
            # 2 STA on the same R -> capture process
            elif (len(sta_on_R[i][r]) >= 2):
                max_power = 0
                max_power_idx = 0
                power_sum = 0
                for sta_index in sta_on_R[i][r]:
                    power_sum += sta_power[sta_index] / (sta_distance[sta_index] ** gamma)
                    if sta_power[sta_index] / (sta_distance[sta_index] ** gamma) > max_power:
                        max_power = sta_power[sta_index] / (sta_distance[sta_index] ** gamma)
                        max_power_idx = sta_index
                
                SIR = max_power / (power_sum - max_power)
                capture_flag = False
                if SIR > capture_power_ratio:
                    obo_counter[max_power_idx] = 99999
                    counter_delay_slot += (i+1)
                    success_sta += 1
                    capture_flag = True
                    M_i_s[i] += 1
                for sta_idx in sta_on_R[i][r]:
                    if sta_idx == max_power_idx and capture_flag:
                        continue
                    else:
                        M_i_f[i] += 1
                        retrans_counter[sta_idx] += 1
                        if (retrans_counter[sta_idx] < Lmax):
                            obo_counter[sta_idx] = random.randint(0, ocw[retrans_counter[sta_idx]])

    success_probability = success_sta / sta_amount
    access_delay = 0
    if success_sta > 0:
        access_delay = (TXOP * counter_delay_slot) / success_sta
    
    through_put = success_sta / Imax
    utilization = success_sta / (Imax * RA_RU)
    
    average_power = 0
    if success_sta > 0:
        average_power = total_power / success_sta
#     average_power = total_power / sta_amount
    
    return success_probability, access_delay, through_put, utilization, average_power, M_i_s, M_i_f, M_i_1

In [6]:
sample = 1000

# 1, 2, ... 10, 20, 30, ... 100, 200, 300, ... 1000
m_list_simu = np.concatenate([np.arange(1, 11, 1), np.arange(20, 110, 10), np.arange(200, 1100, 100)])
Ps_list_simu = [0 for i in range(28)]
Da_list_simu = [0 for i in range(28)]
T_list_simu = [0 for i in range(28)]
U_list_simu = [0 for i in range(28)]
power_list_simu = [0 for i in range(28)]

MiS_total = [0 for _ in range(Imax)]
MiF_total = [0 for _ in range(Imax)]
Mi_1_total = [0 for _ in range(Imax)]

# R = 148
# CR = 1.5
step = 4
R_list = [9, 18, 37, 74, 148]
# R_list = [148]
CR_list = [3]
# CR_list = [1.5, 3, 6, float('inf')]
for CR in CR_list:
    print(f'------------ CR={CR}--------------')
    for R in R_list:
#         print(f'------------ R={R}--------------')
        for idx, m in enumerate(m_list_simu):
            success_probability_total = 0;
            access_delay_total = 0;
            through_put_total = 0;
            utilization_total = 0;
            no_success_counter = 0;
            power_usage_total = 0;
            for _ in range(sample):
                success_probability, access_delay, through_put, utilization, power_usage, M_i_s, M_i_f, M_i_1 = simulation(sta_amount=m, RA_RU=R, Lmax=5, TXOP=5.673, ocw_min=7, ocw_max=31, CR=CR, step=step, gamma=2, r=5)
                
                success_probability_total += success_probability
                access_delay_total += access_delay
                through_put_total += through_put
                utilization_total += utilization
                power_usage_total += power_usage
                if success_probability == 0:
                    no_success_counter += 1
                    
                MiS_total = [x + y for x, y in zip(MiS_total, M_i_s)]
                MiF_total = [x + y for x, y in zip(MiF_total, M_i_f)]
                M_i_1_total = [x + y for x, y in zip(M_i_1_total, M_i_1)]
                

            Ps = success_probability_total / sample
            if (sample - no_success_counter > 0):
                Da = access_delay_total / (sample - no_success_counter)
            T = through_put_total / sample
            U = utilization_total / sample
#             power = power_usage_total / (sample - no_success_counter)
            power = power_usage_total / sample
            M_i_s = [x / sample for x in MiS_total]
            M_i_f = [x / sample for x in MiF_total]
            M_i_1 = [x / sample for x in M_i_1_total]

            Ps_list_simu[idx] = Ps
            Da_list_simu[idx] = Da
            T_list_simu[idx] = T
            U_list_simu[idx] = U
            power_list_simu[idx] = power

            print(M_i_f)
            print(M_i_1)
#         print(f'ps_list_simu_{R}_ramp = {Ps_list_simu}')
#         print(f'da_list_simu_{R}_ramp = {Da_list_simu}')
#         print(f't_list_simu_{R}_ramp = {T_list_simu}')
#         print(f'u_list_simu_{R}_ramp = {U_list_simu}')
#         print(f'power_list_simu_{R}_ramp = {power_list_simu}')
#         print(f'ps_list_simu_{R} = {Ps_list_simu}')
#         print(f'da_list_simu_{R} = {Da_list_simu}')
#         print(f't_list_simu_{R} = {T_list_simu}')
#         print(f'u_list_simu_{R} = {U_list_simu}')
#         print(f'power_list_simu_{R} = {power_list_simu}')
#         print("\n")

NameError: name 'Imax' is not defined